In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import torch
from tqdm import tqdm_notebook
from PIL import Image

from torchvision import models
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
from torchvision.models import resnet18,ResNet18_Weights,inception_v3,Inception_V3_Weights
from torch.utils.tensorboard import SummaryWriter

import torch.nn as nn
import torch.utils.data as utils

In [2]:
from IPython.display import display,HTML

In [3]:
display(HTML("<style>.container{ width:100% !important; }</style>"))

In [4]:
df_train = pd.read_csv(r"Data/train.csv")
df_train.head()

,ImageId,EncodedPixels
0,000f6bf48.jpg,131973 1 132228 4 132483 6 132738 8 132993 11 ...
1,002af848d.jpg,290800 6 291055 13 291311 15 291566 18 291822 ...
2,0030401a5.jpg,186833 1 187089 3 187344 6 187600 7 187855 10 ...
3,008621629.jpg,215548 3 215798 9 216051 12 216306 13 216560 1...
4,012f26693.jpg,35258 2 35513 5 35768 8 36023 11 36277 15 3653...


In [5]:
df_train.shape

(781, 2)

In [6]:
df_val = pd.read_csv(r"Data/val.csv")
df_val.head()

,ImageId,EncodedPixels
0,0025bde0c.jpg,315139 8 315395 15 315651 16 315906 17 316162 ...
1,08b40a161.jpg,178410 2 178664 7 178918 11 179172 20 179426 2...
2,1795867f2.jpg,266115 46 266370 51 266625 55 266880 60 267136...
3,190190a6f.jpg,18811 1 19065 5 19320 8 19574 12 19829 15 2008...
4,2fcfb108a.jpg,230653 3 230904 8 231157 11 231413 11 231616 6...


<h2 style="
    background:#1f77b4;
    color:white;
    padding:10px;
    border-radius:8px;
">
RLE decode function
</h2>



In [7]:
def rleDecode(mask_rle,shape=(256,1600)):
    if pd.isna(mask_rle):
       return np.zeros(shape,dtype=np.uint8)
    s = mask_rle.split()
    starts = np.array(s[0::2],dtype=int)-1
    pos    = np.array(s[1::2],dtype=int)
    ends   = starts+pos

    img = np.zeros(shape[0]*shape[1],dtype=np.uint8)
    for start_pos,end_pos in zip(starts,ends):
         img[start_pos:end_pos] = 1
    return img.reshape(256,1600,order='F')
          

<h2 style="
    background:#1f77b4;
    color:white;
    padding:10px;
    border-radius:8px;
">
Dataset Function
</h2>


In [8]:
class SteelDataset(Dataset):
    def __init__(self,image_dir,csv_path):
         self.csv_path = csv_path
         self.df = pd.read_csv(csv_path)
         self.image_dir=image_dir
         self.imageid_unique = self.df.ImageId.unique()

    def __len__(self):
        self.image_len = len(self.imageid_unique)
        return self.image_len

    def __getitem__(self,idx):
        image_id   = self.imageid_unique[idx]
        image_path = os.path.join(self.image_dir,image_id)
        image      = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
        image      = image/255
        image      = torch.tensor(image).unsqueeze(0)
       
        rows        = self.df[self.df.ImageId==image_id]
        mask = np.zeros((256,1600))

        for _,row in rows.iterrows():
            if not pd.isna(row.EncodedPixels):
                mask += rleDecode(row.EncodedPixels)
        mask = np.clip(mask, 0, 1)
        mask = torch.tensor(mask).unsqueeze(0)

        return image.float(), mask.float()

In [9]:
# dataset = SteelDataset(
#     "Data/train_images",
#     "Data/train.csv")
# image,mask=dataset[0]

In [10]:
# image.shape

In [11]:
# mask.shape

In [12]:
train_dataset = SteelDataset("Data/train_images","Data/train.csv")
trainData_loader = DataLoader(dataset=train_dataset,batch_size=4,shuffle=True,drop_last=True)
    

In [13]:
resnet_model = resnet18(weights=ResNet18_Weights.DEFAULT)

<h2>Conv2d calculation</h2>
<img src="image/calc3.png" width="100%">

In [14]:
resnet_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [15]:
resnet_model.fc = nn.Identity()

In [16]:
resnet_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
__BY DOING RESNET ITS SHAPPE BECOME 1,512,8,50 TO INCREASE THE SIZE WE NEED TO UP SAMPLING IT BY DOING DECODER CLASS BELOW THE FINAL SHAPE WILL BECOM 1,1, 

In [21]:
class Decoder(nn.Module):
    def __init__(self):
        super.__init__()
        self.up1 = nn.ConvTranspose2d(512,256,2,2)
        self.up2 = nn.ConvTranspose2d(256,128,2,2)
        self.up3 = nn.ConvTranspose2d(128,64,2,2)
        self.up4 = nn.ConvTranspose2d(64,32,2,2)
        self.up5 = nn.ConvTranspose2d(32,16,2,2)
        self.final = nn.Conv2d(16,1,kernel_size=1)
        # 𝐻 = 128,w = 800, padding=0,output=1 =>1,32,128,800 to 1,1,128,800
    def forward(self,x):
        x = self.up1(x)
        x = self.up2(x)
        x = self.up3(x)
        x = self.up4(x)
        return seld.final(x)

<img src="image/calc.png" width="100%">

<h2>ConvTranspose2d calculation</h2>
<img src="image/calc2.png" width="100%">

In [22]:
class model(nn.Module):
    def __init__(self,resnet_model):
        super.__init__()
        self.resnet_model = resnet_model
        self.Decoder = Decoder()

    def forward(self):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.encoder.layer1(x)
        x = self.encoder.layer2(x)
        x = self.encoder.layer3(x)
        x = self.encoder.layer4(x)

        x = self.decoder(x)
        return x